In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install inline

In [ ]:
pip install Sastrawi

In [ ]:
import inline as inline
import matplotlib
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
import Sastrawi.Stemmer
import string
from nltk.tokenize import word_tokenize
from string import digits
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
fulldataset = pd.read_csv('/kaggle/input/ppkm-sentiment/ppkm_dataset.csv')
testdataset = pd.read_csv('/kaggle/input/ppkm-sentiment/ppkm_test.csv')

In [ ]:
fitur = fulldataset.iloc[:,1].values
labels = fulldataset.iloc[:,0].values
fitur2 = testdataset.iloc[:,1].values
labels2 = testdataset.iloc[:,0].values

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def preprocessing(data):
    fitur_ekstraksi0 = []
    for cuitan in range(0, len(data)):
        tmp = str(data[cuitan]).lower()
        fitur_ekstraksi0.append(tmp)

    fitur_ekstraksi1 = []
    for cuitan in range(0, len(fitur_ekstraksi0)):
        tmp = fitur_ekstraksi0[cuitan].translate(str.maketrans(' ', ' ', digits))  # membuang karakter angka
        fitur_ekstraksi1.append(tmp)

    fitur_ekstraksi2 = []
    for cuitan in range(0, len(fitur_ekstraksi1)):
        tmp = fitur_ekstraksi1[cuitan].translate(str.maketrans(' ', ' ', string.punctuation))  # membuang karakter
        fitur_ekstraksi2.append(tmp)

    fitur_ekstraksi3 = []
    # fitur_ekstraksi = []
    for cuitan in range(0, len(fitur_ekstraksi2)):
        tmp = re.sub(r'\W', ' ',str(fitur_ekstraksi2[cuitan])) # membuang karakter khusus selain angka dan huruf
        tmp = re.sub(r'\s+[a-zA-Z]\s+', ' ',str(fitur_ekstraksi2[cuitan])) # membuang kata yang hanya satu huruf
        tmp = re.sub(r'\^[a-zA-Z]\s+', ' ',str(fitur_ekstraksi2[cuitan])) # membuang kata yang hanya satu huruf dari awal
        tmp = re.sub(r'\s+', ' ',str(fitur_ekstraksi2[cuitan])) # mengganti spasi ganda dengan spasi tunggal
        fitur_ekstraksi3.append(tmp)

    fitur_ekstraksi5 = []
    for cuitan in range(0, len(fitur_ekstraksi3)):
        tmp = word_tokenize(str(fitur_ekstraksi3[cuitan]))
        fitur_ekstraksi5.append(tmp)

    return fitur_ekstraksi5

In [ ]:
stopwordp1 = open('/kaggle/input/ppkm-sentiment/stopwordv1.txt', 'r')
stopwordp2 = stopwordp1.read()
stopwordp = word_tokenize(stopwordp2)

In [ ]:
def swr(a, b):
    filtered_sentence = []
    for w in a:
        if w not in b:
            filtered_sentence.append(w)
    return filtered_sentence

In [ ]:
callbackvalue = preprocessing(fitur)
callbackvalue1 = preprocessing(fitur2)

In [ ]:
def stopw(datanext):
    fitur_ekstraksiku = []
    for cuitan in range(0, len(datanext)):
        tmp = swr(datanext[cuitan], stopwordp)
        fitur_ekstraksiku.append(tmp)
    return fitur_ekstraksiku

fitur_ekstraksinext = stopw(callbackvalue)
fitur_ekstraksinext2 = stopw(callbackvalue1)
print(fitur_ekstraksinext2[1:2])
print('\n')

In [ ]:
def identity_tokenizer(text):
    return text

from sklearn.feature_extraction.text import TfidfVectorizer

vektor_kata = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)
fitur_ekstraksisiap = vektor_kata.fit_transform(fitur_ekstraksinext).toarray()

fitur_ekstraksi = []
for cuitan in range(0, len(callbackvalue)):
    tmp = fitur_ekstraksisiap[cuitan]
    fitur_ekstraksi.append(tmp)

fitur_ekstraksitest = []
for cuitan in range(len(callbackvalue), len(fitur_ekstraksisiap)):
    tmp = fitur_ekstraksisiap[cuitan]
    fitur_ekstraksitest.append(tmp)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC

scores = []
best_svr = SVC(kernel='linear',C=1.0)
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(fitur_ekstraksisiap):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = fitur_ekstraksisiap[train_index], fitur_ekstraksisiap[test_index], labels[train_index], labels[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))
print(scores)
print(np.mean(scores))

In [ ]:
from sklearn.svm import LinearSVC
scores = []
best_svr = LinearSVC(max_iter=1000,loss='squared_hinge')
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(fitur_ekstraksisiap):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = fitur_ekstraksisiap[train_index], fitur_ekstraksisiap[test_index], labels[train_index], labels[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))
print(scores)
print(np.mean(scores))

In [ ]:
from sklearn.svm import LinearSVC
scores = []
best_svr = MLPClassifier(solver='adam', hidden_layer_sizes=(100,),activation='logistic',
                    verbose=1, random_state=None)
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(fitur_ekstraksisiap):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = fitur_ekstraksisiap[train_index], fitur_ekstraksisiap[test_index], labels[train_index], labels[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))
print(scores)
print(np.mean(scores))

In [ ]:
from sklearn.svm import LinearSVC
scores = []
best_svr = RandomForestClassifier(n_estimators=150, random_state=0)
cv = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in cv.split(fitur_ekstraksisiap):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = fitur_ekstraksisiap[train_index], fitur_ekstraksisiap[test_index], labels[train_index], labels[test_index]
    best_svr.fit(X_train, y_train)
    scores.append(best_svr.score(X_test, y_test))
print(scores)
print(np.mean(scores))